In [ ]:
import vega_sim.grpc.client as vac
from vega_sim.api import data_raw, data
import datetime
import csv

data_client = vac.VegaTradingDataClientV2(f"api1.vega.community:3007")

In [ ]:
start = None
end = None

markets = [
    [
        "LDO_USDT-PERP",
        "603f891b390fa67ac1a7b8f520c743e776cf58da7b8637e2572d556ba55f2878",
    ]
]

for market in markets:
    trades = data.get_trades(
        data_client=data_client,
        market_id=market[1],
        market_price_decimals_map={market[1]: 4},
        market_position_decimals_map={market[1]: 1},
        market_asset_decimals_map={market[1]: 6},
        start=start,
        end=end,
    )

    with open(f"./{market[0]}_trade_data_start_{start}_end_{end}.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(
            [
                "timestamp",
                "trade_id",
                "datetime",
                "trade_type",
                "size",
                "price",
                "buyer",
                "seller",
                "market_trading_mode",
                "auction_start",
                "auction_end",
                "best_bid_volume",
                "best_bid_price",
                "best_offer_volume",
                "best_offer_price",
                "mark_price",
                "internal_twap",
                "external_twap",
                "funding_payment",
                "funding_rate",
            ]
        )
        s = None
        e = None
        for p in reversed(trades):
            s = datetime.datetime.utcfromtimestamp(
                p.timestamp * 1e-9
            ) - datetime.timedelta(seconds=10)
            e = datetime.datetime.utcfromtimestamp(
                p.timestamp * 1e-9
            ) - datetime.timedelta(seconds=5)
            market_data = data.get_market_data_history(
                data_client=data_client,
                market_id=market[1],
                start=s,
                end=e,
            )

            r = [
                p.timestamp,
                p.id,
                datetime.datetime.utcfromtimestamp(p.timestamp * 1e-9).strftime(
                    "%Y-%m-%d %H:%M:%S %z"
                ),
                p.trade_type,
                p.size,
                p.price,
                p.buyer,
                p.seller,
            ]

            if len(market_data) >= 1:
                md = market_data[-1]
                r.append(md.market_trading_mode)
                if md.auction_start.year > 1975:
                    r.append(md.auction_start)
                else:
                    r.append("")
                if md.auction_end.year > 1975:
                    r.append(md.auction_end)
                else:
                    r.append("")
                r.append(md.best_bid_volume)
                r.append(md.best_bid_price)
                r.append(md.best_offer_volume)
                r.append(md.best_offer_price)
                r.append(md.mark_price)

                pd = md.product_data.perpetual_data
                if not pd is None:
                    r.append(pd.internal_twap)
                    r.append(pd.external_twap)
                    r.append(pd.funding_payment)
                    r.append(pd.funding_rate)
                else:
                    r.append("")
                    r.append("")
                    r.append("")
                    r.append("")

                for lp in sorted(
                    md.liquidity_provider_fee_share, key=lambda x: x.party
                ):
                    r.append(lp)
                for lp in sorted(md.liquidity_sla, key=lambda x: x.party):
                    r.append(lp)

            writer.writerow(r)

In [ ]:
start = datetime.datetime(year=2024, month=2, day=14, hour=21, minute=20)
end = datetime.datetime(year=2024, month=2, day=17, hour=8, minute=20)

for market in markets:
    funding_periods = data.list_funding_periods(
        data_client=data_client, market_id=market[1], start=start, end=end
    )
    with open(f"./{market[0]}_funding_periods.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(
            [
                "start_time",
                "end_time",
                "internal TWAP",
                "external TWAP",
                "funding payment",
                "funding rate",
            ]
        )
        for p in funding_periods:
            # we want completed periods only
            if not p.end_time is None:
                writer.writerow(
                    [
                        p.start_time.strftime("%Y-%m-%d %H:%M:%S %z"),
                        p.end_time.strftime("%Y-%m-%d %H:%M:%S %z"),
                        p.internal_twap,
                        p.external_twap,
                        p.funding_payment,
                        p.funding_rate,
                    ]
                )

In [ ]:
oracle_spec_id = "dd08b862c2d9ddcb6f29e9ea6a93cfc3093f1d12a259ebcffef096344cc7448d"

oracle_data = data_raw.list_oracle_data(
    data_client=data_client,
    oracle_spec_id=oracle_spec_id,
)

with open(f"./oracle_id_{oracle_spec_id}_data.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["broadcast_at", "name", "value"])
    for p in oracle_data:
        timestamp = datetime.datetime.fromtimestamp(
            p.external_data.data.broadcast_at * 1e-9, tz=timezone("UTC")
        )
        date = timestamp.strftime("%Y-%m-%d %H:%M:%S %z")
        data_point = p.external_data.data.data[0]
        writer.writerow([date, data_point.name, data_point.value])